In [28]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import math
import csv
from csv import writer
import concurrent.futures
import random

In [ ]:
#scrape from freeproxy.world
url = "https://www.freeproxy.world/"
proxylist =[]
#filter through all pages
for i in range(1,75):
    print(i)
    try:
        #filter to http proxy
        querystring = {"type":"http","anonymity":"","country":"","speed":"","port":"","page":f"{i}"}

        payload = ""
        headers = {"User-Agent": "insomnia/8.4.5"}

        #site request
        site = requests.request("GET", url, data=payload, headers=headers, params=querystring)
        soup = BeautifulSoup(site.content, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs:
            try:
                ip = tr.find('td',class_=re.compile('show-ip-div')).text.strip()
                port = tr.find('a').text.strip()
                proxylist.append(f'{ip}:{port}')
            except:
                pass
    except:
        pass
print(len(proxylist))




#scrape free proxies from proxyscrape.com
headers = {"User-Agent": "insomnia/8.4.5"}
#site request
url = 'https://proxyscrape.com/free-proxy-list'
site = requests.get(url,headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape http proxy download link
divs = soup.find_all('div', class_=re.compile('itemcard downloadcard'))
for div in divs:
    h2 = div.find('h2')
    try:
        if(h2.text.strip()=='HTTP Proxies'):
            a = div.find('a')
            download = a['download_url']
    except:
        pass

#get content of download link
try:
    site = requests.get(download,headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    #add proxies to testlist
    tempproxylist = soup.text.strip().split('\r')
    for proxy in tempproxylist:
        proxylist.append(proxy)

    print(len(proxylist))
except:
    pass    
#scrape proxies from free-proxy-list.net
url = 'https://free-proxy-list.net/'
#site + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape table
try:
    table = soup.find('div', class_=re.compile('table-responsive'))
    trlabels = table.find_all('tr')
except:
    pass
#scrape proxies and add to test list
try:
    for i in trlabels:
            tdlabels = i.find_all('td')
            if(tdlabels[6].text.strip()=='yes'):
                proxylist.append(tdlabels[0].text.strip())
except:
    pass
print(len(proxylist))
#scrape proxies from hidemy.io
url = 'https://hidemy.io/en/proxy-list/'
#site request + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
table = soup.find('div',class_=re.compile('table_block'))
#scrape table
trlabels = table.find_all('tr')
#scrape proxies and add to test list
for i in trlabels:
    tdlabels = i.find_all('td')
    try:

        if(tdlabels[4].text.strip()=='HTTP'):
            proxylist.append(tdlabels[0].text.strip())
    except:
        pass
print(len(proxylist))





#test proxies
workingProxies = []
def extract(proxy):
    works = False
    try:
        r = requests.get('https://www.whatismybrowser.com/detect/what-is-my-ip-address', proxies={'http':proxy, 'https':proxy}, timeout=2)
        soup = BeautifulSoup(r.content, 'html.parser')
        div = soup.find('div', class_='detected_result')
        ip = div.find('div').text.strip()
        print(f'{r}, {ip}')
        works = True
        workingProxies.append(proxy)
        with open('working_proxies.csv', 'w', encoding='UTF8', newline='') as workingProx:
            writer = csv.writer(workingProx)
            for k in workingProxies:
                writer.writerow([k])

    except:
        print(proxy)
    return works

#excecute test faster
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract, list(set(proxylist)))

In [29]:
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 16


In [30]:
teamID_dict = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

In [31]:
url = 'https://www.landofbasketball.com/results/2021_2022_scores.htm'

payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)

<Response [200]>


In [32]:
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all('div', class_=re.compile('left margen-b8'))
print(links[1])

<div class="left margen-b8" style="width: 120px ; height: 60px">
<span><a href="2021_2022_nov_scores.htm">November</a></span>
<br>
					225 Games
				</br></div>


In [33]:
monthLinks = []
for i in range(len(links)):
    a = links[i].find('a')
    part = a.get('href')
    monthLinks.append(part)
print(len(monthLinks))
print(monthLinks)

9
['2021_2022_oct_scores.htm', '2021_2022_nov_scores.htm', '2021_2022_dec_scores.htm', '2021_2022_jan_scores.htm', '2021_2022_feb_scores.htm', '2021_2022_mar_scores.htm', '2021_2022_apr_scores.htm', '2021_2022_may_scores.htm', '2021_2022_jun_scores.htm']


In [34]:
gameLinks = []
for part in monthLinks:    
    url = f'https://www.landofbasketball.com/results/{part}'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('div', class_=re.compile('margin-t1 margen-b2 clearfix'))
    for i in range(len(links)):
        a = links[i].find('a')
        part = a.get('href')
        gameLinks.append(part)
print(f'Game Links found: {len(gameLinks)}')

Game Links found: 1323


In [35]:
#initalize csv
head = ['date', 'homeTeam', 'homeTeam_id', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 
        'homeTeam_FG', 'homeTeam_3P', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 
        'homeTeam_TO', 'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 
        'awayTeam_points_q4', 'awayTeam_FG', 'awayTeam_3P', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 
        'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'winner']

with open('nbaDatabase.csv', 'w', encoding='UTF8', newline='') as nbaDatabase:
    writer = csv.writer(nbaDatabase)
    writer.writerow(head)

In [36]:
for part in gameLinks:
    #access next game
    url = f'https://www.landofbasketball.com/results/{part}'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

    #soup
    soup = BeautifulSoup(response.content, 'html.parser')

    #initialize variables
    date = None
    awayTeam = None
    homeTeam = None
    awayTeam_id = None
    homeTeam_id = None
    awayTeam_points_total = None
    homeTeam_points_total = None
    awayTeam_points_q1 = None
    homeTeam_points_q1 = None
    awayTeam_points_q2 = None
    homeTeam_points_q2 = None
    awayTeam_points_q3 = None
    homeTeam_points_q3 = None
    awayTeam_points_q4 = None
    homeTeam_points_q4 = None
    awayTeam_FG = None
    homeTeam_FG = None
    awayTeam_3P = None
    homeTeam_3P = None
    awayTeam_FT = None
    homeTeam_FT = None
    awayTeam_O_Reb = None
    homeTeam_O_Reb = None
    awayTeam_D_Reb = None
    homeTeam_D_reb = None
    awayTeam_Total_Reb = None
    homeTeam_Total_Reb = None
    awayTeam_Ast = None
    homeTeam_Ast = None
    awayTeam_Stl = None
    homeTeam_Stl = None
    awayTeam_Blk = None
    homeTeam_Blk = None
    awayTeam_TO = None
    homeTeam_TO = None
    awayTeam_PF = None
    homeTeam_PF = None
    winner = None

    #scrape+clean date
    dateHome = soup.find('div', class_=re.compile('pad-b10 pad-l5'))
    dateHome = dateHome.text.strip().split('\n')
    date = dateHome[0].split(': ')
    date = date[1]

    #scrape+clean homeTeam
    homeTeam = dateHome[1].split(': ')
    homeTeam = homeTeam[1]

    table = soup.find('table', class_=re.compile('size-11 a-left'))
    tableText = table.text.strip().split('\n')
    for i in range(len(tableText)):
        newI = tableText[i].replace('\t', '')
        tableText[i] = newI

    filtered_text = [item for item in tableText if item]
    filtered_text.pop(2)
    team1 = filtered_text[:2]
    team2 = filtered_text[2:]

    #scrape+clean awayTeam and point totals
    if homeTeam in team1:
        homeTeam_points_total = team1[0]
        awayTeam = team2[1]
        awayTeam_points_total = team2[0]
    else:
        homeTeam_points_total = team2[0]
        awayTeam = team1[1]
        awayTeam_points_total = team1[0]

    #extract team ids from dict
    homeTeam_id = teamID_dict[f'{homeTeam}']
    awayTeam_id = teamID_dict[f'{awayTeam}']

    #scrape + clean points per quarter
    quarterTable = soup.find('table', re.compile('by pad-y5 margen-b5'))
    quarterStats = quarterTable.find_all('td')
    qStatList = []
    for stat in quarterStats:
        qStatList.append(stat.text.strip())
    team1qStats = qStatList[6:12]
    team2qStats = qStatList[12:]
    if homeTeam_id in str(team1qStats[0]):
        homeTeam_points_q1 = team1qStats[1]
        homeTeam_points_q2 = team1qStats[2]
        homeTeam_points_q3 = team1qStats[3]
        homeTeam_points_q4 = team1qStats[4]
        awayTeam_points_q1 = team2qStats[1]
        awayTeam_points_q2 = team2qStats[2]
        awayTeam_points_q3 = team2qStats[3]
        awayTeam_points_q4 = team2qStats[4]
    if awayTeam_id in str(team1qStats[0]):
        homeTeam_points_q1 = team2qStats[1]
        homeTeam_points_q2 = team2qStats[2]
        homeTeam_points_q3 = team2qStats[3]
        homeTeam_points_q4 = team2qStats[4]
        awayTeam_points_q1 = team1qStats[1]
        awayTeam_points_q2 = team1qStats[2]
        awayTeam_points_q3 = team1qStats[3]
        awayTeam_points_q4 = team1qStats[4]

    #scrape + clean all other stats
    statTables = soup.find_all('div', class_=re.compile('clearfix pad-y5'))
    for i in range(len(statTables)):
        team1 = statTables[i].find('h3', class_=re.compile('d-inline-block a-middle size-11 margen-t0 margen-b0 negri'))
        if awayTeam in team1.text.strip():
            awayTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
            awayTeamStats = awayTeamStats.text.strip().split('\n')
            awayTeam_FG = awayTeamStats[3]
            awayTeam_3P = awayTeamStats[4]
            awayTeam_FT = awayTeamStats[5]
            awayTeam_O_Reb = awayTeamStats[6]
            awayTeam_D_Reb = awayTeamStats[7]
            awayTeam_Total_Reb = awayTeamStats[8]
            awayTeam_Ast = awayTeamStats[9]
            awayTeam_Stl = awayTeamStats[10]
            awayTeam_Blk = awayTeamStats[11]
            awayTeam_TO = awayTeamStats[12]
            awayTeam_PF = awayTeamStats[13]
        if homeTeam in team1.text.strip():
            homeTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
            homeTeamStats = homeTeamStats.text.strip().split('\n')
            homeTeam_FG = homeTeamStats[3]
            homeTeam_3P = homeTeamStats[4]
            homeTeam_FT = homeTeamStats[5]
            homeTeam_O_Reb = homeTeamStats[6]
            homeTeam_D_Reb = homeTeamStats[7]
            homeTeam_Total_Reb = homeTeamStats[8]
            homeTeam_Ast = homeTeamStats[9]
            homeTeam_Stl = homeTeamStats[10]
            homeTeam_Blk = homeTeamStats[11]
            homeTeam_TO = homeTeamStats[12]
            homeTeam_PF = homeTeamStats[13]

    #extract winner
    home_points = int(re.search(r'\d+', homeTeam_points_total).group())
    away_points = int(re.search(r'\d+', awayTeam_points_total).group())
    if home_points > away_points:
        winner = homeTeam
    elif away_points > home_points:
        winner = awayTeam
    else:
        winner = "Draw"



    print(f'Scraping {homeTeam} vs {awayTeam}: {date}...')
    #write game to csv

    gameStats = [
        [date, homeTeam, homeTeam_id, homeTeam_points_total, homeTeam_points_q1, homeTeam_points_q2, homeTeam_points_q3, homeTeam_points_q4,
        homeTeam_FG, homeTeam_3P, homeTeam_O_Reb, homeTeam_D_Reb, homeTeam_Total_Reb, homeTeam_Ast, homeTeam_Stl, homeTeam_Blk, 
        homeTeam_TO, homeTeam_PF, awayTeam, awayTeam_id, awayTeam_points_total, awayTeam_points_q1, awayTeam_points_q2, awayTeam_points_q3,
        awayTeam_points_q4, awayTeam_FG, awayTeam_3P, awayTeam_O_Reb, awayTeam_D_Reb, awayTeam_Total_Reb, awayTeam_Ast, awayTeam_Stl, 
        awayTeam_Blk, awayTeam_TO, awayTeam_PF, winner],
    ]

    with open('nbaDatabase.csv', 'a', encoding='UTF8', newline='') as nbaDatabase:
        writer = csv.writer(nbaDatabase)      
        writer.writerows(gameStats) 


Scraping Milwaukee Bucks vs Brooklyn Nets: October 19, 2021...
Scraping Los Angeles Lakers vs Golden State Warriors: October 19, 2021...
Scraping Charlotte Hornets vs Indiana Pacers: October 20, 2021...
Scraping Detroit Pistons vs Chicago Bulls: October 20, 2021...
Scraping New York Knicks vs Boston Celtics: October 20, 2021...
Scraping Toronto Raptors vs Washington Wizards: October 20, 2021...
Scraping Memphis Grizzlies vs Cleveland Cavaliers: October 20, 2021...
Scraping Minnesota Timberwolves vs Houston Rockets: October 20, 2021...
Scraping New Orleans Pelicans vs Philadelphia 76ers: October 20, 2021...
Scraping San Antonio Spurs vs Orlando Magic: October 20, 2021...
Scraping Utah Jazz vs Oklahoma City Thunder: October 20, 2021...
Scraping Phoenix Suns vs Denver Nuggets: October 20, 2021...
Scraping Portland Trail Blazers vs Sacramento Kings: October 20, 2021...
Scraping Atlanta Hawks vs Dallas Mavericks: October 21, 2021...
Scraping Miami Heat vs Milwaukee Bucks: October 21, 2021..